In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver    
import time
import pandas as pd
import re
import collections
import nltk
from nltk import word_tokenize, Text, FreqDist
from nltk.corpus import wordnet as wn
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
wnl = nltk.WordNetLemmatizer()
import pathlib
from newspaper import Article
from urllib.parse import urlparse
# importing itertools for accumulate()
import itertools
# importing functools for reduce()
import functools
from tqdm.notebook import tqdm_notebook
import json
import praw
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import os
import concurrent.futures
import time
from tqdm import tqdm
import gc
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from datetime import datetime

In [3]:
from urllib.parse import urlparse

domain = urlparse('http://www.example.test/foo/bar').netloc
print(domain) # --> www.example.test

www.example.test


In [4]:
'''

Input : Xpath for the dropdown list 
Output: Xpath for the choices in the dropdown

'''
def getNewsContentByArticle(Article, url):
    # get content 
    try:
        content = Article(url)
        content.download()
        content.parse()
        content = content.text
    except Exception as e:
        print(e)
        content = ""
    return content

def getNewsContentByGoogle(title):
    
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
        
    channels = ["channelnewsasia", "straitstime","todayonline"]
    
    # to search
    query = "news:" + title
 
    content, domain, url = "", "", ""
    
    for j in search(query, tld="co.in", num=10, stop=10, pause=2):
        url = j
        
        domain = urlparse(j).netloc
        
        isIn = [True for c in channels if c in domain]
        
        content = getNewsContentByArticle(Article,j)
        
        if not isIn == []:
            break
    
    return {'content':content, 'domain':domain,'url':url}
        

def getPostListings(driver, xpath_items):
    try:
        items = driver.find_elements("xpath", xpath_items)
    except Excpetion as e:
        print('\n Empty post listing due to ',e)
        items = []
        pass
    return items

def getDropdownChoices(driver, xpath_dropdown, xpath_choices):
    # Wait for initialize, in seconds
    wait = WebDriverWait(driver, 10)

    dropdown = wait.until(EC.visibility_of_element_located((By.XPATH, xpath_dropdown)))
    
    time.sleep(1)
    
    dropdown.click()
    
    #2. find the choices on the list
    try:
        dropdown = driver.find_elements("xpath",xpath_choices)
    except ElementClickInterceptedException:
        print('\n-- No dropdown list found --')
        pass
    
    return dropdown

def getChildElement(node, xpath):
    
    xpath = ".//descendant-or-self::" + xpath
    
    try:
        child_node = node.find_element("xpath", xpath)
    except NoSuchElementException:
        print("\n-- Unable to find the child element")
        raise
        
    return child_node


def goNextPage(driver, xpath):
    try:
        button = driver.find_element("xpath",xpath)
        button.click()
        time.sleep(1)
    except:
        #print('\n-- No element {} found --'.format(xpath))
        raise

In [5]:
'''
one_instance = {
    'p_id': post_id,
    'title': title,
    'p_url':title_url,
    'domain':domain,
    'datetime': datetime,
    'scores': scores,
    'no_of_cmts': no_of_cmts,
    'article': content,
    'c_url': cmt_url,
    'comments': comments
}
'''

"\none_instance = {\n    'p_id': post_id,\n    'title': title,\n    'p_url':title_url,\n    'domain':domain,\n    'datetime': datetime,\n    'scores': scores,\n    'no_of_cmts': no_of_cmts,\n    'article': content,\n    'c_url': cmt_url,\n    'comments': comments\n}\n"

In [6]:
xpath_post_listings = "//table[contains(@id,'thread')]/tbody[not(contains(@class, 'emptb'))]"

## Links Grab Test 

In [7]:
xpath_title = ".//descendant-or-self::th[contains(@class,'new') or contains(@class,'common')]//a[contains(@class, 'xstt')]" #post title

xpath_author = ".//descendant-or-self::td[contains(@class,'by')]//a"

xpath_last_modified_attr = ".//descendant-or-self::td[contains(@class,'byy')]/em//span[boolean(@title)]" #

xpath_last_modified_text = ".//descendant-or-self::td[contains(@class,'byy')]/em/span" #

xpath_no_of_cmts = ".//descendant-or-self::td[contains(@class,'num')]/a"

xpath_views = ".//descendant-or-self::td[contains(@class,'num')]/span[not(string-length(@*))]"

xpath_url = ".//descendant-or-self::td[contains(@class,'icn')]/a" #the url for the post and associated comments

xpath_main_page_next = "//div[contains(@class, 'pg')]//a[contains(@class, 'nxt')]"

xpath_sub_page_next = "//div[contains(@class, 'pg')]//a[contains(@class, 'nxt')]"


def getPostListingItems(post_listing):
    try:
        title = post_listing.find_element("xpath", xpath_title).text
    except Exception as e:
        print('\n--',e)
        title = ''
    
    try:
        url = post_listing.find_element("xpath", xpath_url).get_property('href')
    except Exception as e:
        url = ''        
    
    try:
        author = post_listing.find_element("xpath", xpath_author).text
    except Exception as e:
        author = ''
        
    try:
        last_modified_1 = post_listing.find_element("xpath", xpath_last_modified_text).text
    except Exception as e:
        last_modified_1 = ''
        
    try:
        last_modified_2 = post_listing.find_element("xpath", xpath_last_modified_attr).get_property('title')
    except Exception as e:
        last_modified_2 = ''
        
    date = last_modified_1 +'T'+ last_modified_2
    
    try:
        no_of_cmts = post_listing.find_element("xpath", xpath_no_of_cmts).text
    except Exception as e:
        no_of_cmts = ''
    
    try:
        views = post_listing.find_element("xpath", xpath_views).text
    except Exception as e:
        views = ''
        
    #print('\n--DEBUG:',title,url,author,no_of_cmts,views,date)
    
    return {
        'title':title.strip(),
        'p_url':url.strip(),
        'author':author.strip(),
        'no_of_cmts':no_of_cmts.strip(),
        'views': views.strip(),
        'datetime': re.sub('[\u4E00-\u9FFF]+','',date.strip())
    }
def _getNewsContentByGoogle(title):
    
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
        
    #channels = ["channelnewsasia", "straitstime","todayonline"]
    
    # to search
    query = '新加坡 '+title 
 
    content, domain, url = "", "", ""
    
    for j in search(query, tld="co.in", num=10, stop=10, pause=5):
        url = j
        
        domain = urlparse(j).netloc
        
        content = getNewsContentByArticle(Article,j)
        
        print('\n--DEBUG-','URL:',j,'DOMAIN:',domain)
        print('\n',content)
    
    return {'content':content, 'domain':domain,'url':url}

In [14]:
url = "https://www.xinjiapoyan.com/forum-jishixinwen-1.html"

linkscache = []

#initalise crawler option(s)
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_argument('headless')
ROOT_DIR = pathlib.Path().absolute()
print(ROOT_DIR)
driver = webdriver.Chrome(executable_path=(str(ROOT_DIR)+"/chromedriver"), options=options)
driver.set_window_size(1120, 1000)  
driver.get(url)
time.sleep(2)

SEARCHING = True

C:\Users\liux5\Documents\Project 3 - DSTA


C:\Users\liux5\AppData\Local\Temp\ipykernel_21940\1787981826.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=(str(ROOT_DIR)+"/chromedriver"), options=options)


In [8]:
BEGIN_FROM = datetime.strptime('2021-12-30', '%Y-%m-%d')

END_AT = datetime.strptime('2022-12-31', '%Y-%m-%d')

count = 0
out_of_range = 0
threshold = 500

while SEARCHING:
    items = getPostListings(driver, xpath_post_listings)
    for item in items:
        out_items = getPostListingItems(item)
        if out_items['author'] == '新加坡眼编辑' and int(out_items['no_of_cmts']) > 0:
            try:
                date = [each for each in out_items['datetime'].split('T') if each.find('-')!=-1][0]
                date = datetime.strptime(date, '%Y-%m-%d')
                out_items['date'] = date
            except Exception as e:
                print('\n null date attribute:', e)
                continue
            if date >= BEGIN_FROM and date <= END_AT:
                linkscache.append(out_items)
                count = count + 1
                print('\n\t-- {} reocords have been collected on SG-Eye.'.format(count))

            else:
                if date < BEGIN_FROM:
                    out_of_range = out_of_range + 1
                    print('\n\t-- DEBUG: date out of range (BEFORE) ', date, out_of_range)
                else:
                    print('\n\t-- DEBUG: date out of range (AFTER) ', date)     
            if out_of_range >= 500:
                SEARCHING = False
                break
        else:
            pass #do nothing
    try:
        goNextPage(driver, xpath_main_page_next)
    except Exception as e:
        SEARCHING = False

C:\Users\liux5\Documents\Project 3 - DSTA


C:\Users\liux5\AppData\Local\Temp\ipykernel_14716\1794983621.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=(str(ROOT_DIR)+"/chromedriver"), options=options)



	-- DEBUG: date out of range (AFTER)  2023-02-01 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-15 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-15 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-07 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-14 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-08 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-15 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-14 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-13 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-12 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-14 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-15 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-14 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-14 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-14 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-14 00:00:00

	-- DEBUG: date out of range (AFTER)  2023-02-14 00:00:

In [15]:
import pickle

def save_pickle_object(obj, file_path):
    with open(file_path, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
        
def load_pickle_object(file_path):
    with open(file_path, "rb") as input_file:
        data = pickle.load(input_file)
        return data

linkscache = load_pickle_object('data/SG_EYE_LINKS.pikle')

In [16]:
linkscache

[{'title': '在新加坡，Singpass密码忘记了怎么办？',
  'p_url': 'https://www.xinjiapoyan.com/thread-42369-1-5.html',
  'author': '新加坡眼编辑',
  'no_of_cmts': '4',
  'views': '1725',
  'datetime': '2022-12-27T',
  'date': datetime.datetime(2022, 12, 27, 0, 0)},
 {'title': '饮食没“低钠”？新加坡政府：摄入盐还得摄入“严”！',
  'p_url': 'https://www.xinjiapoyan.com/thread-42368-1-6.html',
  'author': '新加坡眼编辑',
  'no_of_cmts': '7',
  'views': '1836',
  'datetime': '2022-12-27T',
  'date': datetime.datetime(2022, 12, 27, 0, 0)},
 {'title': '新加坡曾有条“黑街”，时常上演生死大片...',
  'p_url': 'https://www.xinjiapoyan.com/thread-42070-1-6.html',
  'author': '新加坡眼编辑',
  'no_of_cmts': '7',
  'views': '1773',
  'datetime': '2022-12-20T',
  'date': datetime.datetime(2022, 12, 20, 0, 0)},
 {'title': '“短期在新加坡，哪种电话卡上网流量多，费用最划算？”',
  'p_url': 'https://www.xinjiapoyan.com/thread-42340-1-6.html',
  'author': '新加坡眼编辑',
  'no_of_cmts': '6',
  'views': '2098',
  'datetime': '2022-12-26T',
  'date': datetime.datetime(2022, 12, 26, 0, 0)},
 {'title': '最新入境新加坡攻略来了！来

In [17]:
xpath_posts = "//div[contains(@id, 'postlist')]//div[contains(@id,'post_') and not(contains(@id,'post_rate') or contains(@id,'post_new'))]" #the content of the post

out_comments = []

for item in tqdm(linkscache):
    driver.get(item['p_url'])    
    #print('\n-- DEBUG: post_url - ',item['p_url'])
    count = 0
    comments = []    
    while True:
        cmts = getPostListings(driver,xpath_posts)
        for post in cmts:
            #print(item['p_url'])
            #c_id
            xpath_c_id = ""
            try:
                c_id = post.get_attribute('id').split('_')[-1]
            except Exception as e:
                c_id = 'n/a'
            #print('-- DEBUG: c_id :',c_id)            
            #p_id
            xpath_p_id = ".//descendant-or-self::div[contains(@class, 'pct')]//div[contains(@class, 'quote')]//a"
            
            pattern_p_id = "pid=([0-9]+)"
            
            try:
                c_p_id = post.find_element("xpath", xpath_p_id).get_property('href')
                match =  re.search(pattern_p_id, c_p_id, re.IGNORECASE)

                if match :
                    c_p_id = match.group(1)
                
            except Exception as e:
                c_p_id = ""
                
            #print('-- DEBUG: parent_id :',c_p_id)
            
            #datetime
            xpath_c_datetime = ".//descendant-or-self::div[contains(@class, 'pti')]/div[contains(@class, 'authi')]"
            
            try:
                c_datetime = post.find_element("xpath", xpath_c_datetime).text
                
                c_datetime = re.sub(r'[\u4e00-\u9fff]+|\|','',c_datetime)
                
                c_datetime = re.sub(r'\s+',' ',c_datetime.strip())
                
                if c_datetime == "" or len(c_datetime) < 4:
                    c_datetime = post.find_element("xpath", xpath_c_datetime+"//span").get_attribute('title')
            #print('-- DEBUG: datetime :',c_datetime)                     
                
            except Exception as e:
                c_datetime = "n/a"
            #print('-- DEBUG: datetime :',c_datetime) 
            
            #user
            xpath_c_user = ".//descendant-or-self::div[contains(@class, 'pi')]/div[contains(@class, 'authi')]/a"
            try:
                c_user = post.find_element("xpath", xpath_c_user).text      
            except Exception as e:
                c_user = "n/a"         
            #print('-- DEBUG: user :',c_user)   
            
            #comment
            xpath_comment = ".//descendant-or-self::div[contains(@class,'pcb')]"
            try:
                c_text = post.find_element("xpath", xpath_comment).text
                c_text = re.sub('来自: (Android|iPhone)客户端','',c_text)
                c_text = re.sub('\s+', ' ', c_text.strip())
            except Exception as e:
                #print('\n-- DEBUG: something wrong with scraping comments, ',e)
                c_text = 'n/a'
            #print('-- DEBUG: text :',c_text)             
            
            comments.append({
                'cmt_id' : c_id,
                'cmt_replyTo' : c_p_id,
                'cmt_user': c_user,
                'cmt_text': c_text,
                'cmt_score': '',
                'cmt_datetime': c_datetime            
            })

        try:
            goNextPage(driver, xpath_main_page_next)
        except:
            #print('\n-- Scraping completed for post:',item['title'])
            break
    out_comments.append(comments)


100%|████████████████████████████████████████████████████████████████████████████| 2550/2550 [2:35:29<00:00,  3.66s/it]


In [18]:
count = 0
for i in range(len(linkscache)):
    linkscache[i]['comments'] = out_comments[i]
    count = count + len(out_comments[i])

print(len(linkscache))


2550
44990


In [31]:
import pickle

with open('data/SG_Eye-1-12.json', 'w', encoding='utf-8') as output_file:
    json.dump(out_comments , output_file ,indent = 2, default=str, ensure_ascii=False)

In [34]:
print(len(linkscache))
print(len(out_comments))

2550
2550


In [74]:
SG_EYE_DF = pd.DataFrame()
url, date_publish, news_title, news_article, comments = [], [], [], [], []

for idx, each in enumerate(out_comments):
    try:
        article = each[0]
        url.append(linkscache[idx]['p_url'])
        news_title.append(linkscache[idx]['title'])
        date_publish.append(article['cmt_datetime'])
        news_article.append(article['cmt_text'])
        comments.append(each[1:])
    except Exception as e:
        print(e)

    

SG_EYE_DF['url'] = url
SG_EYE_DF['datetime'] = date_publish
SG_EYE_DF['article'] = news_article
SG_EYE_DF['news_title'] = news_title
SG_EYE_DF['lang'] = 'CN'
SG_EYE_DF['org'] = 1
SG_EYE_DF['news_category'] = ''
SG_EYE_DF['new_comments'] = comments
SG_EYE_DF['source'] = 5

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [76]:
SG_EYE_DF
save_pickle_object(SG_EYE_DF, "news_crawler/output/df/SG_EYE_DF_1-12.pickle")

def load_pickle_object(file_path):
    with open(file_path, "rb") as input_file:
        data = pickle.load(input_file)
        return data
    
SG_EYE_DF = load_pickle_object("news_crawler/output/df/SG_EYE_DF_1-12.pickle")

In [69]:
for idx, row in tqdm(SG_EYE_DF.iterrows(), total=SG_EYE_DF.shape[0]):
    
    comments = str(row["new_comments"])

    comments = ast.literal_eval(comments)

    comments_df = pd.DataFrame.from_records(comments)

    comments_df['source'] = row['source']

    comments_df['lang'] = row['lang']
    
    display(comments_df)
    
    break

  0%|                                                                                         | 0/2545 [00:00<?, ?it/s]

,cmt_id,cmt_replyTo,cmt_user,cmt_text,cmt_score,cmt_datetime,source,lang
0,356423,,璞玉缘,刚去联络所办完，很简单 回复,,2022-12-27 12:16:01,5,CN
1,356718,,流浪的小哥,联络所一分钟搞定 回复,,2022-12-28 16:36:10,5,CN
2,356831,,Jia嘉,网上更改也是一分钟搞定 回复,,2022-12-29 05:38:32,5,CN
3,365795,,sgajmd,问题是习惯刷脸之后，密码早就忘得一干二净。 回复,,2023-2-12 17:05:34,5,CN


  0%|                                                                                         | 0/2545 [00:00<?, ?it/s]
